In [1]:
from __future__ import division
import sys
import os
import time
import glob
import PIL.Image as Image
import numpy as np

import theano
import theano.tensor as T

import lasagne
#import lasagne.layers.dnn

import Cae

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 750 Ti (CNMeM is disabled, cuDNN 5110)


In [2]:
def im2ar(x):
    y = [[[0]*len(x)]*len(x)]*3
    y[0] = x[:,:,0]
    y[1] = x[:,:,1]
    y[2] = x[:,:,2]
    return y

def ar2im(x):
    y = [[[0]*3]*len(x)]*len(x)
    y[:,:,0] = x[0]*255 
    y[:,:,1] = x[1]*255
    y[:,:,2] = x[2]*255
    return y

In [3]:
def load_dataset_mscoco():
    #Path
    mscoco="/home/myuser/Documents/Lasagne/inpainting"
    split="train2014"
    data_path = os.path.join(mscoco, split)
    imgs = glob.glob(data_path + "/*.jpg")
    
    X_train = []
    y_train = []
            
    for i, img_path in enumerate(imgs):
        img = Image.open(img_path)
        img_array = np.divide(np.array(img,dtype='float32'),255)

        if len(img_array.shape) == 3:
            temp = np.copy(img_array)
            input = np.copy(img_array)
            input[16:48, 16:48,:] = 0
            target = img_array[16:48, 16:48,:]
        else:
            input[:,:,0] = np.copy(img_array)
            input[:,:,1] = np.copy(img_array)
            input[:,:,2] = np.copy(img_array)
            target = input[16:48, 16:48,:]
            input[16:48, 16:48,:] = 0
        
        X_train.append(im2ar(input))
        y_train.append(im2ar(target))
    
    split="val2014"
    data_path = os.path.join(mscoco, split)
    imgs = glob.glob(data_path + "/*.jpg")
    
    X_val = []
    y_val = []
            
    for i, img_path in enumerate(imgs):
        img = Image.open(img_path)
        img_array = np.divide(np.array(img,dtype='float32'),255)

        if len(img_array.shape) == 3:
            input = np.copy(img_array)
            input[16:48, 16:48,:] = 0
            target = img_array[16:48, 16:48,:]
        else:
            input[:,:,0] = np.copy(img_array)
            input[:,:,1] = np.copy(img_array)
            input[:,:,2] = np.copy(img_array)
            target = input[16:48, 16:48,:]
            input[16:48, 16:48,:] = 0
        
        X_val.append(im2ar(input))
        y_val.append(im2ar(target))
    
    # We reserve the last 10000 training examples for testing.
    X_val, X_test = X_val[:-10000], X_val[-10000:]
    y_val, y_test = y_val[:-10000], y_val[-10000:]
    
    return (np.array(X_train),np.array(y_train),np.array(X_val),np.array(y_val),np.array(X_test),np.array(y_test))

In [4]:
# Main
def main():
    
    # Hyper Params
    num_epochs = 100
    learning_rate = 0.01
    momentum = 0.975
    batchsize = 200
    
    # Load the dataset
    print("Loading data...")
    X_train, y_train, X_val, y_val, X_test, y_test = load_dataset_mscoco()
    
    # Prepare Theano variables for inputs and targets
    inputVar = T.tensor4('inputs')
    target_var = T.tensor4('targets')

    # Build Network
    #print("Building model and compiling functions...")
    network = Cae.build(inputVar)

    # Training Loss expression
    prediction = lasagne.layers.get_output(network)
    loss = lasagne.objectives.squared_error(prediction, target_var)
    loss = loss.mean()
    # Add regularization lasagne.regularization.

    # Update expressions 
    # Stochastic Gradient Descent (SGD) with Nesterov momentum
    params = lasagne.layers.get_all_params(network, trainable=True)
    updates = lasagne.updates.nesterov_momentum(loss, params, learning_rate, momentum)

    # Test Loss expression
    # 'Deterministic = True' disables droupout
    test_prediction = lasagne.layers.get_output(network, deterministic=True)
    test_loss = lasagne.objectives.squared_error(test_prediction,target_var)
    test_loss = test_loss.mean()

    # Train Function
    train_fn = theano.function([inputVar, target_var], loss, updates=updates)

    # Test Function
    val_fn = theano.function([inputVar, target_var], test_loss)
    
   
    #Load model
    with np.load('cae-1000e.npz') as f:
        param_values = [f['arr_%d' % i] for i in range(len(f.files))]
    lasagne.layers.set_all_param_values(network, param_values)
    
    # Training Loss expression
    prediction = lasagne.layers.get_output(network)
    predict = theano.function([inputVar], prediction)
    


    #show net
    #PrintLasagneNetInfo(network)
    
    # Show prediction
    print("Predicting test set")
    Y = predict(X_test[0:100])
    return (X_test[0:100],Y)

In [5]:
def PrintLasagneNetInfo(lasagneNetwork): 
    import nolearn
    import lasagne
    nolearnNetwork = nolearn.lasagne.NeuralNet( 
          layers=lasagneNetwork, 
          update=lasagne.updates.adam, 
          objective_loss_function= lasagne.objectives.squared_error
          ) 
    nolearnNetwork.initialize() 
    nolearn.lasagne.PrintLayerInfo()(nolearnNetwork)

In [6]:
X,Y=main()

Loading data...
Network output: (None, 16, 56, 56)
Network output: (None, 16, 28, 28)
Network output: (None, 16, 26, 26)
Network output: (None, 16, 13, 13)
Network output: (None, 16, 14, 14)
Network output: (None, 16, 28, 28)
Network output: (None, 3, 32, 32)
network params 11973059
Predicting test set


In [7]:
X[:,:,16:48,16:48]=Y[:,:,:,:]

In [8]:
from PIL import Image
im = Image.fromarray((X.reshape(10,10,3,64,64).transpose(1,3,0,4,2)*255).astype('uint8').reshape(10*64, 10*64,3))
im.save("your_file4.jpeg")